# 3D Morphable Model Tutorial

#### 0. importing libraries and data

In [ ]:
import os
import time
import copy
import math
import pickle
import statistics
import numpy as np
import pandas as pd
import open3d as o3d
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from summer2022_toolbox.visualization_3D_objects import *
from summer2022_toolbox.preprocessing import *
from summer2022_toolbox.read_object import *
from summer2022_toolbox.model_averaging import *
from summer2022_toolbox.model_PCA import *
from summer2022_toolbox.morphable_model import *
from summer2022_toolbox.model_evaluation import *

        The dataset we are using in this tutorial is ModelNet40: Princeton's free 3D Object dataset that contains CAD models from 40 categories.

#### 1. Introduction to 3D Morphable Models

The cornerstone of 3D Morphable Models were first proposed by Blanz and Vetter in 1999. It represents a 3D model with a linear combination of examplar shapes. According to Blanz and Vetter's work, there are three types of variations for facial 3DMMs: A shape model that captures geometric variations, an expression model that captures facial expressions, and an appearance model that captures variations in textures and illuminations. In this tutorial, we will only focus on the most basic shape model, which is a global shape model proposed in the same work of Blanz and Vetter, to help readers to capture the basic ideas of 3D Morphable Models. 

Morphable model requries full correspondence within the dataset. It means that each component in one shape represent the position of the same point in all shapes. Achieveing this full correspondence is the main challenge in building a morphable model. In the dataset we are using (ModelNet40), each 3D shape has different size, orientation, and number of points. Thus, our first step will discuss how do we unify those features. But we will leave the unifying of number of points and achieving full correspondence in Step 2, as it can be achieved when calculating the average shape. 

Now we have full correspondence within the dataset, we can describe any new shape $S_{model}$ as a linear combination of the m examplar shapes $S$:  

$$
S_{model} = \sum_{i=1}^{m}a_{i}S_{i}
$$

This algorithm can be optimized by representing the unique parts of each examplar shape $S_{i}$ as a sum of an average model $\overline{S}$ and a linear combination of eigenvectors $s_{i}$: 

$$
S_{i} = \overline{S} + \alpha_{i}s_{i}
$$

We can pick the first k dominating eigenvectors use Principle Component Analysis (PCA). Once we finished calculating the average model and the eigenvectors, we have the morphable model to describe or create any new shape $S_{model}$ with a linear function:

$$
S_{model} = \overline{S} + \sum_{i=1}^{m - 1}\alpha_{i}s_{i}
$$

## Step 1: Preprocessing

### 1. Scale and Downsampling data

First, we need to unify the size of all 3D shapes to a target size. We also need to downsample the shape to reduce our cost of calculation.

In [ ]:
#test
filename = "ModelNet40/car/train/car_0001.off"
source = read_pointcloud(filename)
#scale to a target size, downsample with specified voxel size
source_down, source_fpfh = prepare_point_cloud(source, target_size=1000, voxel_size=20)
source_points = np.asarray(source.points).T #(3, n_points) data
source_down_points = np.asarray(source_down.points).T

print("Original number of points: ", source_points.shape[1])
print("Number of points after downsampling: ", source_down_points.shape[1])

'''
Visualization of scaling and downsampling results
'''
# draw3DPoints(source_points, title = "Before scaling and downsampling")
# draw3DPoints(source_down_points, title = "After scaling and downsampling")

### 2. Unifying orientation

The 3D Morphable model requires a full correspondence between all shapes. In other words, it requires that the points which represent the "left front tire" of any car should be matched to the "left front tire" of all other cars in the dataset. Thus, to match the points from one model to the corresponding points on another shape, we must have them aligned in same orientation. 

In [ ]:
#Read data
f1 = "ModelNet40/car/train/car_0001.off"
car_1 = read_pointcloud(f1)
#scale to a target size, downsample with specified voxel size
car_1, car_1_fpfh = prepare_point_cloud(car_1, target_size=1000, voxel_size=20)
car_1 = np.asarray(car_1.points).T

f2 = "ModelNet40/car/train/car_0003.off"
car_2 = read_pointcloud(f2)
#scale to a target size, downsample with specified voxel size
car_2, car_2_fpfh = prepare_point_cloud(car_2, target_size=1000, voxel_size=20)
car_2 = np.asarray(car_2.points).T
comparePointClouds(car_1, car_2, title = "Source: car_1, Target: car_2")

Here, we can see that car_0001.off is facing $-y$ direction, car_0003.off is facing $-x$ direction. If we try to find the matching points of "left front tire" on car_0003 for car_0001, we would end up getting the "right front tire" of car_0003 being matched to car_0001's "left front tire". We must move car_0003's center to match the center of car_0001, and rotate car_0003 $90^{\circ}$ counterclockwise.

##### (i) Manually orient cars

In [ ]:
#move to same center
car_1_center = car_1.mean(axis = 1).reshape(3,1)
car_1 = car_1 - car_1_center
car_2_center = car_2.mean(axis = 1).reshape(3,1)
car_2 = car_2 - car_2_center
comparePointClouds(car_1, car_2, title = "Source: car_1, Target: car_2")

In [ ]:
#rotate to same direction
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(car_2.T)
R = pcd.get_rotation_matrix_from_xyz((0, 0, np.pi / 2))
pcd.rotate(R, center=(0,0,0))
car_2 = np.asarray(pcd.points).T

In [ ]:
comparePointClouds(car_1, car_2, title = "Source: car_1, Target: car_2")

##### (ii) Registration algorithms (Ransac + ICP) to orient cars

However, when dealing with large amount of similar 3D models, use registration algorithms is a faster way to orient most cars to same direction. Registration algorithms are usually used to match different captures of same point cloud object. It computes an optimized transformation matrix that maximizes the fitness between the source object and the reference object.

According to Open3D's Global Registration Tutorial, a conventional way of matching objects with different orientation is to perform Global Registration algorithms first, and use its resulting transformation matrix as the initial transformation matrix required by Local Registration algorithms for refined alignment.

The global registration algorithm we are applying is Ransac. Its parameters are empirical value provided by [Choi, 2015] according to the Open3D documentation. The local refinement algorithm we are using is ICP registration. More details please check [Open3D's Global Registration Tutorial](http://www.open3d.org/docs/release/tutorial/pipelines/global_registration.html)

In [ ]:
f1 = "ModelNet40/car/train/car_0001.off"
car_1 = read_pointcloud(f1)
#scale to a target size, downsample with specified voxel size
car_1, car_1_fpfh = prepare_point_cloud(car_1, target_size=1000, voxel_size=20)

f2 = "ModelNet40/car/train/car_0003.off"
car_2 = read_pointcloud(f2)
#scale to a target size, downsample with specified voxel size
car_2, car_2_fpfh = prepare_point_cloud(car_2, target_size=1000, voxel_size=20)

comparePointClouds(np.asarray(car_1.points).T, 
                   np.asarray(car_2.points).T, title = "Before Registration: Source: car_1, Target: car_2")

ICP_threshold = 15
target_size=1000
voxel_size=20

source_down, source_fpfh = prepare_point_cloud(car_1, target_size, voxel_size)
target_down, target_fpfh = prepare_point_cloud(car_2, target_size, voxel_size)

GCP_transformation = GCP_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
ICP_transformation = ICP_registration(source_down, target_down, ICP_threshold, GCP_transformation)
print("Fitness score: {:.2%}".format(ICP_transformation.fitness))

source_temp = copy.deepcopy(source_down)
source_temp.transform(ICP_transformation.transformation)

comparePointClouds(np.asarray(source_temp.points).T, 
                   np.asarray(target_down.points).T, title = "After Registration: Source: car_1, Target: car_2")

We can see that the result is as good as manually orientation. We can also repeat this registration process multiple times to maximize the fitness score between source model and reference model.

In [ ]:
transformations = []
scores = []
for i in range(10):
    GCP_transformation = GCP_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
    ICP_transformation = ICP_registration(source_down, target_down, ICP_threshold, GCP_transformation)
    transformations.append(ICP_transformation)
    scores.append(ICP_transformation.fitness)
    
max_idx = scores.index(max(scores))
Max_Transformation = transformations[max_idx]
print("Fitness score: {:.2%}".format(Max_Transformation.fitness))

In [ ]:
source_temp = copy.deepcopy(source_down)
source_temp.transform(Max_Transformation.transformation)

comparePointClouds(np.asarray(source_temp.points).T, 
                   np.asarray(target_down.points).T, title = "After Registration: Source: car_1, Target: car_2")

Even though this process could fail on some unusual shapes (e.g. align an F1 race car with SUV), it is still efficient when dealing with a large amount of data.

#### 3. Preprocess All

In [ ]:
preprocess_and_save(folder = 'ModelNet40/car/',
                    ref_name = 'train/car_0002.off', 
                    save_path = 'data/preprocessed/car/')

## Step 2: Modeling

### 1. Calculating the average model

Now we have preprocessed data that have unified sizes and orientations, we can achieve full correspondence within the dataset. 
First, we can pick a random car as our starting reference shape, and match every point in other cars to the closest point in this reference car. We repeat this process a few times to make sure the average model can represent the entire dataset. The pseudo code is:
    
    for n repetition:
        for every car_i in dataset:
            for every point_j in car_reference:
                find the index of the closest point in car_i to point_j
                save the index

            reorder points in car_i by indexes to match the indexes of car_reference
        calculate car_average
        if the mean squared error between car_average and car_reference is less than threshold:
            break loop
        update car_reference with car_average
        
    return the final average shape

In [ ]:
#load all preprocessed data
f1 = open('data/preprocessed/car/train.txt','rb')
train_X = pickle.load(f1)
f2 = open('data/preprocessed/car/test.txt','rb')
test_X = pickle.load(f2)

#select reference model
S_reg_index = 1
S_reg = train_X[S_reg_index]
draw3DPoints(S_reg, 'Reference Car')

We can save all calculated average models.

In [ ]:
S_mean = train_Mean(train_X, S_reg, 'data/model/car_mean.txt')

In [ ]:
#load mean car
f1 = open('data/model/car_mean.txt','rb')
S_mean_all = pickle.load(f1)
for X in S_mean_all:
    draw3DPoints(X)

### 2. Align all training and testing cars with the average model

Now we have the average shape, we can achieve full correspondence by aligning each shape in the dataset with the average shape by following the same pseudo code in last small step. This would result to a dataset that each object has same number of points and each point represent the same feature in all cars (e.g. left front tire). 

In [ ]:
#load mean car
f1 = open('data/model/car_mean.txt','rb')
S_mean_all = pickle.load(f1)
#pick average model
X_avg = S_mean_all[-1]

In [ ]:
#align train data
train_tilde, train_reorder = reOrder(train_X, S_reg)

In [ ]:
#save aligned train data
aligned_train = open('data/preprocessed/car/train_aligned.txt','wb')
pickle.dump(train_reorder, aligned_train)
#load all aligned cars, if fail save again
f1 = open('data/preprocessed/car/train_aligned.txt','rb')
X_train = pickle.load(f1)

In [ ]:
#align test data
test_tilde, test_reorder = reOrder(test_X, S_reg)

In [ ]:
#save aligned train data, if fail save again
aligned_test = open('data/preprocessed/car/test_aligned.txt','wb')
pickle.dump(test_reorder, aligned_test)
f2 = open('data/preprocessed/car/test_aligned.txt','rb')
X_test = pickle.load(f2)

### 3. Calculating eigenvectors with PCA

Our data is currently in shape (3, n). In order to calculate the eigenvectors from the entire datset, we can flatten our data and represent it with a shape-vector $S = (x_{1}, y_{1}, z_{1}, x_{2}, ..., x_{n}, y_{n}, z_{n}) \in \mathbb{R}^{3n}$. 

In [ ]:
#load all aligned cars
f1 = open('data/preprocessed/car/train_aligned.txt','rb')
X_train = pickle.load(f1)
f2 = open('data/preprocessed/car/test_aligned.txt','rb')
X_test = pickle.load(f2)

#load average model
average_models = open('data/model/car_mean.txt','rb')
mean_Xs = pickle.load(average_models)
X_avg = mean_Xs[-1]
X_avg_flat = X_avg.flatten('F')

In [ ]:
# flatten data from (3, n) to (3n, 1)
X_train_flat = pointcloud_to_flat(X_train)
X_test_flat = pointcloud_to_flat(X_test)

##### PCA get eigenvectors

Singular value decomposition (SVD) can factorize a matrix into three seperate matrices  $U$, $D$, and $V$. Here, our input matrix is in shape $(m, 3n)$ where $m$ is the number of objects in the dataset and $n$ is the number of 3d coordinate in each object. SVD would decompose this matrix of data into $U$ that its column vectors spans $\mathbb{R}^{m}$, $D$ that is a diagonal matrix of the square root of nonzero eigenvalues of our input matrix, and $V$ that its column vectors spans $\mathbb{R}^{3n}$. Since we only have $m$ objects in our dataset, the nubmer of nonzero eigenvalues generated by SVD should also be $m$. Which means that only the first 193 column vectors of $V$ can represent our dataset. 

In [ ]:
U, D, V = np.linalg.svd(X_train_flat - X_avg_flat)

In [ ]:
plt.clf
plt.plot(D, '.-')
plt.axis('tight')

##### Try to fit model

We know that a new model can be described by this linear function:
$$
S_{model} = \overline{S} + \sum_{i=1}^{m - 1}\alpha_{i}s_{i}
$$
Thus, 
$$
S_{model} - \overline{S} = \sum_{i=1}^{m - 1}\alpha_{i}s_{i} = \mathbf{\alpha}\mathbf{s}
$$
where $\mathbf{\alpha}$ represent the weights of eigenvectors in columns of $\mathbf{s}$.
Since eigenvectors are orthogonal with each other (i.e. $\langle s_{i},s_{j}\rangle_{j \neq i} = 0$, $\langle s_{i},s_{i}\rangle = 1$), we can compute the weights $\mathbf{\alpha}$ by multiplying $S_{model} - \overline{S}$ with $\mathbf{s}$:

$$
(S_{model} - \overline{S}) \mathbf{s} = \sum_{i=1}^{m - 1}\sum_{j=1}^{m - 1}\alpha_{i}\langle s_{i},s_{j}\rangle = \mathbf{\alpha}
$$


In [ ]:
#fit model with test shape
carMM = MorphableModel(X_avg, V, len(D))

i=3
fitted_model = carMM.fit(X_test_flat[i])
fitted_model_3D = single_flat_to_pointcloud(fitted_model)

In [ ]:
print("Mean Squared Error between original model and average model: %.4f"%(((X_test_flat[i] - X_avg_flat)**2).mean()))
print("Mean Squared Error between original model and fitted model: %.4f"%(((X_test_flat[i] - fitted_model)**2).mean()))
draw3DPoints(X_test[i], 'Original test shape')
draw3DPoints(fitted_model_3D, 'Fitted test shape')
draw3DPoints(X_avg)

We can observe from the above plots that the linear function can already capture the shape of our test car. The fitted shape is a vague silhouette since this is a global shape morphable model and we only have 191 data. The details in the fitted shape can be improved if we apply local shape morphable model which requires a segmentation on the shape before computing the linear generator function for morphable model on different parts such as windshield, tires, or seats.

## Step 3: Evaluating Results

#### Plot Mean Squared Error vs. Number of Features

In order to define an optimized number of features between $0$ and $m$(number of objects in training set), we can plot Mean Squared Error between a test shape and the shape fitted onto it with different number of features.

In [ ]:
plot_MSE_n_feature(X_test_flat, V, X_avg, len(D))

We can observe from the plot that the first 50 features reduced most of the errors. It means that most of the variations in the shapes can be explained by the first 50 features. Thus, we can examine these features and check what do they actually control in creating a new car shape from the morphable model.

In [ ]:
## 
weights_all = []
for i_fet in range(len(D)):
    features = V[:,i_fet]
    w = (X_test_flat - X_avg.flatten('F'))@features
    weights_all.append(w)

In [ ]:
#we only examine first 50 eigenvectors
draw3DpointsSlider(V, 0, X_avg, weights_all)

#### Now is time to create your own car with morphable model!

In [ ]:
new_car = create_new_car(V, X_avg, weights_all)